In [1]:
%run -i "../util/util_simple_classifier.ipynb"
%run -i "../util/file_utils.ipynb"

In [3]:
# Import necessary libraries
import re
from openai import OpenAI
from sklearn.metrics import classification_report

In [4]:
client = OpenAI()

In [5]:
train_dataset = load_dataset("SetFit/bbc-news", split="train")
test_dataset = load_dataset("SetFit/bbc-news", split="test")

In [13]:
example = test_dataset[0]['text']
category = test_dataset[0]['label_text']  # 0: business, 1: entertainment, 2: politics, 3: sport, 4: tech
print(example)
print(category)

carry on star patsy rowlands dies actress patsy rowlands  known to millions for her roles in the carry on films  has died at the age of 71.  rowlands starred in nine of the popular carry on films  alongside fellow regulars sid james  kenneth williams and barbara windsor. she also carved out a successful television career  appearing for many years in itv s well-loved comedy bless this house. rowlands died in hove on saturday morning  her agent said.  born in january 1934  rowlands won a scholarship to the guildhall school of speech and drama scholarship when she was just 15.  after spending several years at the players theatre in london  she made her film debut in 1963 in tom jones  directed by tony richardson. she made her first carry on film in 1969 where she appeared in carry on again doctor. rowlands played the hard-done-by wife or the put-upon employee as a regular carry on star. she also appeared in carry on at your convenience  carry on matron and carry on loving  as well as othe

In [12]:
prompt="""You are classifying texts by topics. There are 5 topics: tech, entertainment, business, politics and sport. 
Output the topic and nothing else. For example, if the topic is business, your output should be "business".
Give the following text, what is its topic from the above list without any additional explanations: """ + example
response = client.chat.completions.create(
    model="gpt-5",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)
print("Predicted topic:", response.choices[0].message.content.strip())

Predicted topic: entertainment


In [14]:
# Define a function to generate predictions using GPT
def get_gpt_classification(input_text):
    prompt="""You are classifying texts by topics. There are 5 topics: tech, entertainment, business, politics and sport.
    Output the topic and nothing else. For example, if the topic is business, your output should be "business".
    Give the following text, what is its topic from the above list without any additional explanations: """ + input_text
    response = client.chat.completions.create(
        model="gpt-5",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    predicted_topic = response.choices[0].message.content.lower().strip()
    return predicted_topic

In [19]:
test_df = test_dataset.to_pandas()
test_df = test_df.sample(frac=1, random_state=42) 
test_data = test_df[0:5].copy() # limit to 5 samples for demonstration
test_data["gpt_prediction"] = test_data["text"].apply(get_gpt_classification)

In [20]:
# Define a function that deals with potential issues with GPT outputs
def get_one_word_match(input_text):
    loc = re.search(
        r'tech|entertainment|business|politics|sport', input_text).span()
    return input_text[loc[0]:loc[1]]
test_data["gpt_prediction_cleaned"] = test_data["gpt_prediction"].apply(get_one_word_match)

In [22]:
label_list = ['tech', 'business', 'sport', 'entertainment', 'politics']
test_data["gpt_label"] = test_data["gpt_prediction_cleaned"].apply(lambda x: label_list.index(x))
print(test_data)

                                                  text  label label_text  \
521  blogger grounded by her airline a us airline a...      0       tech   
737  us interest rate rise expected us interest rat...      1   business   
740  dutch bank to lay off 2 850 staff abn amro  th...      1   business   
660  warnings on woeful wi-fi security companies ar...      0       tech   
411  tsunami to cost sri lanka $1.3bn sri lanka fac...      1   business   

    gpt_prediction gpt_prediction_cleaned  gpt_label  
521           tech                   tech          0  
737       business               business          1  
740       business               business          1  
660           tech                   tech          0  
411       business               business          1  
